In [7]:
import pandas as pd, sqlite3

conn = sqlite3.connect("../data/checking-logs.sqlite")

In [8]:
## Creating datamart table with JOIN
query = """
CREATE TABLE IF NOT EXISTS datamart AS
SELECT
    c.uid,
    c.labname,
    MIN(c.timestamp) AS first_commit_ts,
    MIN(p.datetime) AS first_view_ts
FROM checker c
LEFT JOIN pageviews p ON c.uid = p.uid
WHERE
    c.status = 'ready'
    AND c.numTrials = 1
    AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND c.uid LIKE 'user_%'
GROUP BY c.uid, c.labname;
"""
conn.execute(query)
conn.commit()

In [9]:
## Loading datamart into DataFrame and converting dates
datamart = pd.read_sql("SELECT * FROM datamart;", conn)
datamart["first_commit_ts"] = pd.to_datetime(datamart["first_commit_ts"])
datamart["first_view_ts"] = pd.to_datetime(datamart["first_view_ts"])
datamart.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136


In [10]:
## Division into test and control groups
test = datamart[datamart["first_view_ts"].notna()].copy()
control = datamart[datamart["first_view_ts"].isna()].copy()

In [11]:
## Filling missing values ​​in control
mean_view_ts = test["first_view_ts"].mean()
control["first_view_ts"] = mean_view_ts

In [12]:
## Saving both tables to the DB
test.to_sql("test", conn, if_exists="replace", index=False)
control.to_sql("control", conn, if_exists="replace", index=False)

81

In [13]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [15]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [16]:
conn.close()